## Transfer Learning 을 이용한 Aespa 그룹의 멤버 Winter, karina, Ningning Classifier 생성

  1. Google Image Crawling
  2. Transfer Learning

In [ ]:
# 한글폰트 : 네이버 바른 고딕 설정
# 폰트 설치 완료후에 런타임 다시 시작

!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

In [ ]:
# Drive mount

import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

# 1. 구글 이미지 크롤러를 이용한 학습 이미지 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import urllib.request
import shutil

In [ ]:
# 구글 이미지 크롤링을 위한 드라이버 설치
# 이 부분은 처음 한번만 실행하면 됌.

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
directory_list = [
    '/content/drive/MyDrive/Human_classification/custom_dataset/train/',
    '/content/drive/MyDrive/Human_classification/custom_dataset/val/',
    '/content/drive/MyDrive/Human_classification/custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(idol, train_cnt, val_cnt):

    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + idol):
            try:
                os.makedirs(directory + '/' + idol)
            except:
                print("error")

    # 학습 및 평가 데이터셋 준비하기
    cnt = 1
    for file_name in os.listdir('/content/drive/MyDrive/Human_classification/'+idol):

      # val_cnt 이하는 Validation data로, val_cnt이상, train_cnt이하는 Train data로, 그 외에는 전부 Test data로 사용한다.
        if cnt < val_cnt:
            print(f'[Validation Dataset] {file_name}')
            shutil.move('/content/drive/MyDrive/Human_classification/'+ idol + '/' + file_name, '/content/drive/MyDrive/Human_classification/custom_dataset/val/' + idol + '/' + file_name)
        elif cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move('/content/drive/MyDrive/Human_classification/'+ idol + '/' + file_name, '/content/drive/MyDrive/Human_classification/custom_dataset/train/' + idol + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move('/content/drive/MyDrive/Human_classification/'+ idol + '/' + file_name, '/content/drive/MyDrive/Human_classification/custom_dataset/test/' + idol + '/' + file_name)
        cnt += 1
    shutil.rmtree('/content/drive/MyDrive/Human_classification/'+idol) # 지정된 파일 및 하위 디렉토리 전부 삭제

In [ ]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [ ]:
def crawling_img(name):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=options)
    
    # 해당 url로 이동
    url = "https://www.google.co.kr/imghp?hl=ko&ogbl"
    driver.get(url)

    # 검색창을 찾는 코드
    # 내가 원하는 요소를 클릭하거나 하는 거 찾을 때는 F12를 눌러 왼쪽 위 마우스로
    # 찾는 기능을 이용해서 class, name 을 확인한다.

    elem = driver.find_element_by_name("q")
      # name으로 찾는 방법 외에도, class 등 다른 방법도 사용 가능하다.
      # elem = driver.find_element_by_class("gLFyf gsfi")

    # 원하는 입력값 전송 - 원하는 검색어 검색
    elem.send_keys(name)
    elem.send_keys(Keys.RETURN) # 엔터키 입력

    # 검색 후에 스크롤을 끝까지 다 내려서 일단 다 불러온다.
    SCROLL_PAUSE_TIME = 1

    # 브라우저의 높이를 알아내서 last_height에 저장한다.
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 무한반복문
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # 새로운 높이가 이전의 높이와 같다면 빠져나온다. 즉, 더이상 안내려가면
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click() # 결과더보기 누름
            except:
                break # 결과더보기 없으면 이제 무한반복을 멈춘다.
        last_height = new_height

    # 작은 이미지 클릭
    images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
      # 여러개의 요소를 리스트 해야하므로 elements 사용
      # css의 특성상 클래스의 띄어쓰기에 . 을 대신해서 넣는다.
      # [0].click() 은 가장 첫번째 요소를 끄집어내서 클릭하겠다는 의미

    # Create directory
    dir = "/content/drive/MyDrive/Human_classification/" + name
    createDirectory(dir)

    count = 1
    for image in images:
        try:
          image.click()
          time.sleep(2)
          # 큰 이미지 선택하여 이미지 다운할 수 있는 src 주소 받아오기
          imgUrl = driver.find_element_by_xpath('/html/body/div[3]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img').get_attribute("src")
          # imgUrl = driver.find_element_by_css_selector(".n3VNCb").get_attribute("src")

          # path = '/content/drive/MyDrive/Human_classification/' + name

          urllib.request.urlretrieve(imgUrl, dir + '/' + str(count) + ".jpg")
          # count는 숫자형이기 때문에 str로 string형으로 만들어 문자끼리 더해지게 한다.
          count = count + 1
          if count >= 300:  # 300장 다운 받으면 멈춘다.
              break
        except:
            pass
      
    driver.close()

In [ ]:
idol = '에스파 윈터'
crawling_img(idol)
dataset_split(idol, 270, 30)
# 1~30 : val
# 31~270 : train
# 271~300 : test

In [ ]:
idol = '에스파 카리나'
crawling_img(idol)
dataset_split(idol, 270, 30)

In [ ]:
idol = '에스파 닝닝'
crawling_img(idol)
dataset_split(idol, 270, 30)

#2. Transfer Learning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

# colab에서 GPU사용
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

데이터 가공

In [ ]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의

# transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
# transform.Normalize([R 평균, G 평균, B 평균], [R 표준편차, G 표준편차, B 표준편차])
# ImageNet과 유사할 경우 그 평균과 표준편차를 사용한다. [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = '/content/drive/MyDrive/Human_classification/custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
val_datasets = datasets.ImageFolder(os.path.join(data_dir, 'val'), transforms_val)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('train 데이터셋 크기:', len(train_datasets))
print('val 데이터셋 크기:', len(val_datasets))
print('test 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

이미지 불러보기

In [ ]:
def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()


# 학습 데이터를 배치 단위로 불러오기
iterator = iter(train_dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

Transfer Learning - Last layer train

In [ ]:
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features

# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 3개로 교체하여 마지막 레이어 다시 학습
model.fc = nn.Linear(num_features, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Train

In [ ]:
num_epochs = 50
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in val_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(val_datasets)
    epoch_acc = running_corrects / len(val_datasets) * 100.
    print('[Val Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

Test

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))